In [ ]:
import datetime
import pandas as pd
import plotly.express as px
import dash_table
import jupyter_dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from dataclasses import dataclass, field
from typing import List, Dict, Optional
import random
import numpy
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
import base64
import io

In [ ]:
# ------------------------ Anomalies Class -----------------------
@dataclass
class Anomalies:
    
    # The dataframe
    data: pd.DataFrame = None
    
    # The anomalies names
    anomalies_name_method: Dict[str, str] = field(
        default_factory=lambda: {
            "Erro (Valores Negativos)": "negative_values_error",
            "Estacionariedade": "stationary_values",
            "Deslocamento": "displaced_sensor" 
        }
    )
        
    # indexes from previous simulation
    nve_ps: List = field(default_factory=lambda: [])
    nve_ps_v: List = field(default_factory=lambda: [])
        
    ds_ps: List = field(default_factory=lambda: [])
    sv_ps: List = field(default_factory=lambda: [])
    
    def save_data_with_anomalies(self, csv_name="anomalies.csv"):
        """ Method that saves the dataframe with anomalies """
        self.data.to_csv(path_or_buf=csv_name, index=False)
            
    def add_anomalies_name_and_value_columns(self):
        """ Method that add anomalies columns name and values """
        self.data["anomaly_name"] = ""
        self.data["anomaly_value"] = ""
    
    def negative_values_error(
        self, 
        amount='', 
        persistence_min='', 
        persistence_max=''
    ):
        
        # ------- Remove the previous anomaly simulation --------
        # get the indexes from the previous simulation
        if self.nve_ps:
            self.data.loc[self.nve_ps, "anomaly_name"] = ""
            self.data.loc[self.nve_ps, "anomaly_value"] = ""
        # -------------------------------------------------------    
        
        # Clean self.nve_ps for a new simulation
        self.nve_ps = []
        
        # Check if the arguments are numeric
        if amount.isnumeric():
            for _ in range(int(amount)):
                index = random.randint(0, self.data.shape[0])
                value = random.random() * 10
                self.data.at[index, "anomaly_name"] = "Erro (Valores Negativos)"
                self.data.at[index, "anomaly_value"] = -value
                self.nve_ps.append(index)
                
    def stationary_values(
        self, 
        amount='', 
        persistence_min='', 
        persistence_max=''
    ):
        
        # ------- Remove the previous anomaly simulation --------
        # get the indexes from the previous simulation
        if self.sv_ps:
            self.data.loc[self.sv_ps, "anomaly_name"] = ""
            self.data.loc[self.sv_ps, "anomaly_value"] = ""
        # -------------------------------------------------------    
        
        # Clean self.sv_ps for a new simulation
        self.sv_ps = []
        # Check if the arguments are numeric
        if amount.isnumeric():
#             if persistence_min.isnumeric():
#                 pmin = int(persistence_min)
            
#             if persistence_max.isnumeric():
#                 pmax = int(persistence_max)

#             persistence = random.randint(pmin, pmax)
            persistence = 50
            
            for _ in range(int(amount)):
                index = random.randint(0, self.data.shape[0])
                value = 2.5 #random.random() * 10
                self.data.loc[index:(index + persistence), "anomaly_name"] = "Estacionaridade"
                self.data.loc[index:(index + persistence), "anomaly_value"] = self.data.at[index, "measured"] + value
                self.sv_ps.extend(list(range(index, index + persistence + 1)))
                    
    def displaced_sensor(
        self, 
        amount='',
        persistence_min='',
        persistence_max=''
    ):
        
        # ------- Remove the previous anomaly simulation --------
        # get the indexes from the previous simulation
        if self.ds_ps:
            self.data.loc[self.ds_ps, "anomaly_name"] = ""
            self.data.loc[self.ds_ps, "anomaly_value"] = ""
        # -------------------------------------------------------    
        
        # Clean self.ds_ps for a new simulation
        self.ds_ps = []
        # Check if the arguments are numeric
        if amount.isnumeric():
#             if persistence_min.isnumeric():
#                 pmin = int(persistence_min)
            
#             if persistence_max.isnumeric():
#                 pmax = int(persistence_max)

#             persistence = random.randint(pmin, pmax)
            persistence = 50
            
            for _ in range(int(amount)):
                index = random.randint(0, self.data.shape[0])
                value = 5 #random.random() * 10
                self.data.loc[index:(index + persistence), "anomaly_name"] = "Deslocamento"
                self.data.loc[index:(index + persistence), "anomaly_value"] = self.data.loc[index:(index + persistence), "measured"] + value
                self.ds_ps.extend(list(range(index, index + persistence + 1)))

In [ ]:
def decode_csv_content(csv_content=None, filename=None):
    df = None
    if csv_content:
        content_type, content_string = csv_content.split(',')
        decoded = base64.b64decode(content_string)
        try:
            if 'csv' in filename:
                # Assume that the user uploaded a CSV file
                df = pd.read_csv(
                    io.StringIO(decoded.decode('utf-8'))
                )
            elif 'xls' in filename:
                # Assume that the user uploaded an excel file
                df = pd.read_excel(io.BytesIO(decoded))
        except Exception as e:
            print(e)
            
    return df

In [ ]:
settings = {
    "df_x_column": "datetime",
    "df_y_column": "measured"
}

In [ ]:
# ----------------------- Start Anomalies Class and add the dataframe ---------------
anomalies = Anomalies()

In [ ]:
# ----------------------- Placeholder fig -------------------------------------
placeholder_fig = px.scatter()

In [ ]:
# -------------------- params --------------------------
reference_parameters = {
    "load_csv_n_clicks": 0,
    "injects_anomalies_n_clicks": 0,
    "upload_dataframe_content": "",
    "fig": placeholder_fig
}

In [ ]:
# -------------------------- Tables ------------------------------
anomalies_table = dash_table.DataTable(    
    id='anomalies-table',
    columns=(
        [
            {
                'id': 'Anomalia', 'name': "Anomalia", 'editable': False
            },
            {
                'id': 'Quantidade', 'name': "Quantidade", 'editable': True
            },
            {
                'id': 'Persistência (Min)', 'name': "Persistência (Min)", 'editable': True
            },
            {
                'id': 'Persistência (Max)', 'name': "Persistência (Max)", 'editable': True
            }
        ]
    ),
    data = [ 
        {
            "Anomalia": anomaly_name,
            "Quantidade": "",
            "Persistência (Min)": "", 
            "Persistência (Max)": ""
        }
        for anomaly_name in anomalies.anomalies_name_method
    ]
)

fig_table = dash_table.DataTable(
    
    id='fig-table',
    columns=(
        [
            {
                'id': 'Data', 'name': "Data", 'editable': False
            },
            {
                'id': 'Nível (Original)', 'name': "Nível (Original)", 'editable': False
            },
            {
                'id': 'Tipo de Anomalia', 'name': "Tipo de Anomalia", 'editable': True,
            },
            {
                'id': 'Nível (Anomalia)', 'name': "Nível (Anomalia)", 'editable': True
            }
        ]
    ),
    data=[]
)

In [ ]:
# ------------------- App --------------------------
app = jupyter_dash.JupyterDash(__name__)

In [ ]:
# ------------------- App layout -------------------
app.layout = html.Div([
    anomalies_table,
#     html.Button('Load csv', id='load-dataframe-button', n_clicks=0),
    dcc.Upload(
        id='upload-dataframe',
        children=html.Div(
            [
                html.Button('Load csv', id='load-dataframe-button', n_clicks=0)
            ]
#             [
#             'Drag and Drop or ',
#             html.A('Select Files')
#             ]
        )
    ),
    html.Button('Injects Anomalies', id='injects-anomalies-button', n_clicks=0),
    html.Button('Download csv with Anomalies', id='download-dataframe-with-anomalies-button', n_clicks=0),
    dcc.Graph(
        id='anomalies-fig', 
        figure=placeholder_fig
    ),
    fig_table,
    Download(id="download-anomalies-csv"),    
    html.Div(id='output-data-upload')
])

In [ ]:
# ---------------------------- Select Data display table Callback -----------------------
@app.callback(
    Output('fig-table', 'data'),
    [
        Input('anomalies-fig', 'selectedData')        
    ]
)
def select_data_display_table(selectedData):
    data = []
    if selectedData:
        data = [{
            "Data":anomalies.data.at[point['pointIndex'], "datetime"],
            "Nível (Original)": anomalies.data.at[point['pointIndex'], "measured"],
            "Tipo de Anomalia": anomalies.data.at[point['pointIndex'], "anomaly_name"], 
            "Nível (Anomalia)": anomalies.data.at[point['pointIndex'], "anomaly_value"]
        } for point in selectedData['points']]
    return data

In [ ]:
# ---------------------------- Download Csv with Anomalies  Callback -----------------------
@app.callback(
    Output("download-anomalies-csv", "data"),
    [
        Input('download-dataframe-with-anomalies-button', 'n_clicks')
    ]
)
def download_dataframe_with_anomalies(n_clicks):
    if n_clicks:
        return send_data_frame(anomalies.data.to_csv, filename="anomalies.csv")
    else:
        return None

In [ ]:
# -------------------------- Load CSV and Injects Anomalies Callback ----------------------
@app.callback(
    Output('anomalies-fig', 'figure'),
    [
        Input('load-dataframe-button', 'n_clicks'),
        Input('injects-anomalies-button', 'n_clicks'),
        Input('anomalies-table', 'data'),
        Input('upload-dataframe', 'contents')
    ],
    [
        State('upload-dataframe', 'filename'),
        State('upload-dataframe', 'last_modified')
    ]   
)
def load_csv_and_injects_anomalies(
    load_csv_n_clicks,
    injects_anomalies_n_clicks,
    anomalies_table_data,
    upload_dataframe_content,
    upload_dataframe_filename,
    upload_dataframe_last_modified
):
      
    # ----------------------------- LOAD THE CSV ------------------------------------
    if load_csv_n_clicks != reference_parameters["load_csv_n_clicks"]:
        if upload_dataframe_content:
            if upload_dataframe_content != reference_parameters["upload_dataframe_content"]:
            
                # Load and decode the csv
                df = decode_csv_content(csv_content=upload_dataframe_content, filename=upload_dataframe_filename)
                anomalies.data = df.copy()
                anomalies.add_anomalies_name_and_value_columns()

                # Create a figure for the csv
                fig = px.scatter(df, x=settings["df_x_column"], y=settings["df_y_column"])
                fig.data[0].update(mode='markers+lines', marker={'size': 1})
                fig.update_layout(clickmode='event+select')
                
                # Update Reference Parameters    
                reference_parameters["load_csv_n_clicks"] = load_csv_n_clicks
                reference_parameters["upload_dataframe_content"] = upload_dataframe_content
                reference_parameters["fig"] = fig
    
    # ------------------------ INJECTS ANOMALIES -----------------------------------------
    if injects_anomalies_n_clicks != reference_parameters["injects_anomalies_n_clicks"]:
        if upload_dataframe_content:
            
                # Injects anomalies in the anomlies.data and return the 
                for anomaly in anomalies_table_data:
                    
                    getattr(anomalies, anomalies.anomalies_name_method[anomaly["Anomalia"]])(
                        amount=anomaly["Quantidade"],
                        persistence_min=anomaly["Persistência (Min)"],
                        persistence_max=anomaly["Persistência (Max)"]
                    )
                    
                    # Update the figure values
                    reference_parameters["fig"].data[0].y = numpy.where(
                        anomalies.data["anomaly_value"] != "",
                        anomalies.data["anomaly_value"],
                        anomalies.data[settings["df_y_column"]]
                    )
                                        
                # Update Reference Parameters    
                reference_parameters["injects_anomalies_n_clicks"] = injects_anomalies_n_clicks
                reference_parameters["upload_dataframe_content"] = upload_dataframe_content                
    
    return reference_parameters["fig"]

In [ ]:
# --------------------- MAIN --------------------
if __name__ == '__main__':
#     app.run_server(mode="inline")
    app.run_server()